# Wizards Staff - Quickstart Tutorial

<img src="../img/wizards_staff.png" alt="Wizards Staff logo" width="400"/>

This tutorial provides a comprehensive introduction to using the Wizards Staff package for calcium imaging analysis. By the end of this notebook, you'll understand how to:

1. Set up and initialize Wizards Staff
2. Process calcium imaging data
3. Extract key metrics like rise time, FWHM, and firing rates
4. Generate visualizations
5. Work with and interpret the results

Let's get started!

## 1. Installation and Setup

First, let's install the Wizards Staff package. You can either use pip or install directly from the repository.

In [ ]:
# Uncomment and run this cell if you need to install Wizards Staff
# !pip install git+https://github.com/ArcInstitute/Wizards-Staff.git

Now, let's import the necessary packages:

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import zscore

# Import the main Orb class from Wizards Staff
from wizards_staff import Orb

# Set up matplotlib for better visualizations
plt.style.use('ggplot')
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)

## 2. Understanding the Data Requirements

Before we begin, it's important to understand what data Wizards Staff expects. The package is designed to work with the outputs from calcium imaging pipelines like [Lizard-Wizard](https://github.com/ArcInstitute/Lizard-Wizard) or [CaImAn](https://github.com/flatironinstitute/CaImAn).

### Required Input Files

For each sample, Wizards Staff looks for the following files in your results folder:

- `{sample_name}_dff-dat.npy`: Delta F/F0 (ΔF/F₀) matrix
- `{sample_name}_cnm-A.npy`: Spatial footprints from CaImAn
- `{sample_name}_cnm-idx.npy`: Indices of accepted components
- `{sample_name}_minprojection.tif`: Minimum projection image

### Metadata Format

You also need a metadata CSV file with at least the following columns:
- `Sample`: Unique identifier for each sample, matching filenames
- `Well`: Well identifier (or other grouping variable)
- `Frate`: Frame rate of the recording in frames per second

Let's take a look at a sample metadata file:

In [ ]:
# Sample metadata structure
metadata_example = pd.DataFrame({
    'Sample': ['10xGCaMP-6wk-Baseline-Stream_Stream_F07_s1_FITC_full', 
               '10xGCaMP-6wk-Baseline-Stream_Stream_G03_s1_FITC_full'],
    'Well': ['F07', 'G03'],
    'Frate': [30, 30]
})

print("Example metadata format:")
metadata_example

## 3. Initializing the Orb

The main entry point for Wizards Staff is the `Orb` class. It organizes your data and provides methods for analysis.

Let's initialize an Orb with our data:

In [ ]:
# Set paths to your data
# Update these paths to match your own data locations
results_folder = "path/to/your/calcium_imaging_results"
metadata_path = "path/to/your/metadata.csv"

# Initialize the Orb
orb = Orb(
    results_folder=results_folder,
    metadata_file_path=metadata_path
)

Let's check what the Orb found:

In [ ]:
# Check what samples were found
print(f"Found {len(orb.samples)} samples:")
print(orb.samples)

# Check available data items
print("\nAvailable data items:")
if orb.input_files is not None:
    print(set(orb.input_files['DataItem']))
else:
    print("No input files found.")

## 4. Running a Complete Analysis

The simplest way to use Wizards Staff is to run all analyses at once using the `run_all` method. This will calculate metrics like rise time, FWHM, and firing rates for all samples.

In [ ]:
# Run all analyses at once
orb.run_all(
    group_name="Well",         # Group samples by this metadata column
    frate=30,                  # Frame rate in frames per second
    zscore_threshold=3,        # Threshold for spike detection
    percentage_threshold=0.2,  # Threshold for FWHM calculation
    p_th=75,                   # Percentile threshold for spatial filtering
    min_clusters=2,            # Minimum number of clusters for K-means
    max_clusters=10,           # Maximum number of clusters for K-means
    size_threshold=20000,      # Threshold for filtering out noise
    show_plots=True,           # Show plots during analysis
    save_files=False           # Don't save files in this example
)

## 5. Accessing and Visualizing Results

After running the analysis, we can access the results as pandas DataFrames:

In [ ]:
# Access rise time data
rise_time_df = orb.rise_time_data
if rise_time_df is not None:
    print("Rise Time Data:")
    display(rise_time_df.head())
else:
    print("No rise time data available.")

In [ ]:
# Access FWHM data
fwhm_df = orb.fwhm_data
if fwhm_df is not None:
    print("FWHM Data:")
    display(fwhm_df.head())
else:
    print("No FWHM data available.")

In [ ]:
# Access firing rate data
frpm_df = orb.frpm_data
if frpm_df is not None:
    print("Firing Rate Data:")
    display(frpm_df.head())
else:
    print("No firing rate data available.")

Let's create some basic visualizations of our results:

In [ ]:
# Visualize firing rates by well
if frpm_df is not None:
    plt.figure(figsize=(10, 6))
    # Calculate mean firing rate per well
    well_means = frpm_df.groupby('Well')['Firing Rate Per Min.'].mean().sort_values()
    
    # Create a bar plot
    well_means.plot(kind='bar', color='steelblue')
    plt.title('Average Firing Rate by Well', fontsize=14)
    plt.ylabel('Firing Rate (events/min)', fontsize=12)
    plt.xlabel('Well', fontsize=12)
    plt.xticks(rotation=45)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

## 6. Step-by-Step Analysis

For more control, we can process samples (shards) individually. This allows us to customize the analysis for each sample.

In [ ]:
# Get the first sample
shard = next(orb.shatter())
print(f"Processing sample: {shard.sample_name}")

# Check available files for this sample
print("\nAvailable files:")
for item_name, (file_path, _) in shard.files.items():
    print(f"  {item_name}: {os.path.basename(file_path)}")

In [ ]:
# Apply spatial filtering
filtered_idx = shard.spatial_filtering(
    p_th=75,                # Percentile threshold
    size_threshold=20000,   # Size threshold
    plot=True,              # Show the plot
    silence=False           # Show information
)

print(f"\nFiltered neurons: {len(filtered_idx)} out of {len(shard.get_input('cnm_idx'))}")

In [ ]:
# Convert to calcium signals and spike events
calcium_signals, spike_events = shard.convert_f_to_cs(p=2)

# Z-score the spike events for better detection
zscored_spike_events = zscore(np.copy(spike_events), axis=1)

print(f"Calcium signals shape: {calcium_signals.shape}")
print(f"Spike events shape: {spike_events.shape}")

In [ ]:
# Calculate rise time for filtered neurons
rise_times, rise_positions = shard.calc_rise_tm(
    calcium_signals[filtered_idx],
    zscored_spike_events[filtered_idx],
    zscore_threshold=3
)

# Count events per neuron
events_per_neuron = {idx: len(times) for idx, times in rise_times.items()}
neurons_with_events = sum(1 for count in events_per_neuron.values() if count > 0)

print(f"Detected rise events in {neurons_with_events} out of {len(filtered_idx)} neurons")
print(f"Average events per neuron: {np.mean(list(events_per_neuron.values())):.2f}")

In [ ]:
# Calculate FWHM for filtered neurons
fwhm_pos_back, fwhm_pos_fwd, fwhm_values, spike_counts = shard.calc_fwhm_spikes(
    calcium_signals[filtered_idx],
    zscored_spike_events[filtered_idx],
    zscore_threshold=3,
    percentage_threshold=0.2
)

# Count FWHM events per neuron
fwhm_per_neuron = {idx: len(values) for idx, values in fwhm_values.items()}
neurons_with_fwhm = sum(1 for count in fwhm_per_neuron.values() if count > 0)

print(f"Calculated FWHM for {neurons_with_fwhm} out of {len(filtered_idx)} neurons")

# Calculate average FWHM value
all_fwhm_values = [val for neuron_values in fwhm_values.values() for val in neuron_values]
if all_fwhm_values:
    print(f"Average FWHM: {np.mean(all_fwhm_values):.2f} frames")
else:
    print("No FWHM values found.")

In [ ]:
# Calculate firing rate per minute (FRPM)
frpm_avg, spike_dict = shard.calc_frpm(
    zscored_spike_events,
    filtered_idx,
    frate=30,  # frames per second
    zscore_threshold=3
)

print(f"Average firing rate: {frpm_avg:.2f} events/minute")
print(f"Firing rates range: {min(spike_dict.values()):.2f} to {max(spike_dict.values()):.2f} events/minute")

## 7. Visualizing Neural Activity

Let's create some visualizations of neural activity using Wizards Staff's plotting functions.

In [ ]:
# Import plotting functions
from wizards_staff.plotting import plot_spatial_activity_map, plot_dff_activity, plot_cluster_activity

In [ ]:
# Plot spatial activity map
plot_spatial_activity_map(
    im_min=shard.get_input('minprojection'),
    cnm_A=shard.get_input('cnm_A'),
    cnm_idx=filtered_idx,
    sample_name=shard.sample_name,
    clustering=False,  # Don't color by cluster in this example
    show_plots=True,
    save_files=False
)

In [ ]:
# Plot spatial activity map with clustering
plot_spatial_activity_map(
    im_min=shard.get_input('minprojection'),
    cnm_A=shard.get_input('cnm_A'),
    cnm_idx=filtered_idx,
    sample_name=shard.sample_name,
    clustering=True,  # Color by cluster
    dff_dat=shard.get_input('dff_dat'),  # Required for clustering
    show_plots=True,
    save_files=False
)

In [ ]:
# Plot activity traces
plot_dff_activity(
    dff_dat=shard.get_input('dff_dat'),
    cnm_idx=filtered_idx,
    frate=30,  # frames per second
    sample_name=shard.sample_name,
    max_z=0.6,  # maximum ΔF/F₀ intensity for scaling
    begin_tp=0,  # starting time point
    end_tp=1000,  # ending time point (first 33 seconds at 30 fps)
    show_plots=True,
    save_files=False
)

In [ ]:
# Plot cluster activity
plot_cluster_activity(
    dff_dat=shard.get_input('dff_dat'),
    filtered_idx=filtered_idx,
    sample_name=shard.sample_name,
    min_clusters=2,
    max_clusters=10,
    random_seed=1111111,
    norm=True,  # Normalize activity
    show_plots=True,
    save_files=False
)

## 8. Pairwise Correlation Analysis

Pairwise correlation analysis helps identify functional connectivity between neurons. Let's run this analysis on our data.

In [ ]:
# Run pairwise correlation analysis
orb.run_pwc(
    group_name="Well",  # Group samples by this column
    poly=True,          # Use polynomial fitting
    p_th=75,            # Percentile threshold
    size_threshold=20000,  # Size threshold
    show_plots=True,    # Show plots
    save_files=False    # Don't save files
)

In [ ]:
# Access the pairwise correlation results
pwc_df = orb.df_mn_pwc
intra_df = orb.df_mn_pwc_intra  # Within-group correlations
inter_df = orb.df_mn_pwc_inter  # Between-group correlations

# Display the results
if pwc_df is not None:
    print("Overall Pairwise Correlations:")
    display(pwc_df)
    
    print("\nIntra-group Correlations:")
    display(intra_df)
    
    print("\nInter-group Correlations:")
    display(inter_df)
else:
    print("No pairwise correlation results available.")

## 9. Saving Results

Finally, if not saved during the initial `run_all` function we can save our results to disk for further analysis or sharing.

In [ ]:
# Create output directory
output_dir = "wizards_staff_results"
os.makedirs(output_dir, exist_ok=True)

# Save all results
orb.save_results(
    outdir=output_dir,
    result_names=[
        "rise_time_data", 
        "fwhm_data", 
        "frpm_data", 
        "mask_metrics_data", 
        "silhouette_scores_data",
        "df_mn_pwc", 
        "df_mn_pwc_intra", 
        "df_mn_pwc_inter"
    ]
)

print(f"Results saved to {output_dir}/")

## 10. Conclusion

In this tutorial, we've covered the basics of using Wizards Staff for calcium imaging analysis. We've learned how to:

1. Set up and initialize the Orb with your data
2. Run a complete analysis using `run_all`
3. Process samples step-by-step for more control
4. Calculate important metrics: rise time, FWHM, and firing rates
5. Create visualizations of neural activity
6. Perform pairwise correlation analysis
7. Save and export results

This is just the beginning of what you can do with Wizards Staff. For more advanced usage, check out the [API documentation](../docs/api.md).